In [ ]:
# Posture detection + Voice Warning System (Mac/Windows/Linux)
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 170)  # speaking speed
engine.setProperty('volume', 1.0)  # volume (0.0 to 1.0)

def speak(text):
    """Speak a warning message."""
    engine.say(text)
    engine.runAndWait()

def calculate_angle(a, b, c):
    """Calculate angle between three points (shoulder-hip-knee)."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

def run_posture_detection():
    cap = cv2.VideoCapture(0)

    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert BGR → RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            # Convert back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results.pose_landmarks.landmark

                # Key landmarks
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

                # Calculate angle
                angle = calculate_angle(shoulder, hip, knee)
                
                if angle > 160:
                    posture = "Good Posture"
                    color = (0, 255, 0)
                else:
                    posture = "Bad Posture"
                    color = (0, 0, 255)
                    speak("Warning! Bad posture detected. Please sit straight.")

                # Display status
                cv2.putText(image, posture, (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            except:
                pass

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cv2.imshow("Posture Detection", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

def main():
    print("Starting Posture Detection with Voice Warning... Press 'q' to quit.")
    run_posture_detection()

if __name__ == "__main__":
    main()


Starting Posture Detection with Voice Warning... Press 'q' to quit.


C:\Users\ishaa\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Exception ignored in: <function BSTR.__del__ at 0x000002383AE69E40>
Traceback (most recent call last):
  File "C:\Users\ishaa\AppData\Roaming\Python\Python311\site-packages\comtypes\_post_coinit\bstr.py", line 26, in __del__
    def __del__(self, _free: Callable[["BSTR"], Any] = _SysFreeString) -> None:

KeyboardInterrupt: 


KeyboardInterrupt: 

: 